# Setup

You'll want to set file paths here.

For Xenium data, 

```
wget https://cf.10xgenomics.com/samples/xenium/1.6.0/Xenium_V1_hColon_Non_diseased_Base_FFPE/Xenium_V1_hColon_Non_diseased_Base_FFPE_outs.zip
```

If you need to unzip the spatial/cells file:

`!tar -xzf <file_path_spatial>.gz -C <data_dir>`

(replacing <file_path_spatial> and <data_dir> with the paths to the cells.csv.gz file and the data directory).


In [ ]:
%load_ext autoreload
%autoreload 2

from crispr import Spatial
import scanpy as sc
import squidpy as sq
import os
import pandas as pd

ddd = os.path.join(os.path.expanduser("~"), "visium/spatial/xenium_colon/")
fpd, fps = "Xenium_V1_hColon_Non_diseased_Base_FFPE_outs", "cells.csv"
file_path, file_path_spatial = [os.path.join(ddd, x) for x in [fpd, fps]]

pd.options.display.max_columns = 100


# Object

In [ ]:
self = Spatial(file_path, visium=True, col_cell_type="majority_voting")

# Processing Data

## Preprocessing

In [ ]:
kws_pp = {"kws_hvg": {"min_mean": 0.0125, "max_mean": 10, 
                      "min_disp": 0.5, "flavor": "cell_ranger"}, 
          "target_sum": 10000.0, "cell_filter_pmt": [0, 15], 
          "cell_filter_ngene": [200, None], "cell_filter_ncounts": [3, 40000], 
          "gene_filter_ncell": [3, None], "regress_out": None, 
          "kws_scale": "log", "kws_umap": {"min_dist": 0.3}}
self.preprocess(**kws_pp)

## Clustering

In [ ]:
_ = self.cluster(method_cluster="leiden", 
                 resolution=0.5,
                 kws_pca=dict(n_comps=None, use_highly_variable=True), 
                 kws_umap=dict(min_dist=0.3), 
                 kws_neighbors=None, 
                 kws_cluster=None)

## Annotation

In [ ]:
_ = self.annotate_clusters("Cells_Intestinal_Tract.pkl")

# Analysis

In [ ]:
coords = self.adata.obsm["spatial"]
[coords * sf for sf in [1.0]]

In [ ]:
self.find_cooccurrence()